In [ ]:
from openpyxl import load_workbook
from openpyxl.styles import Alignment

import os
QRM_path = os.path.dirname(os.getcwd()) 
file_path = QRM_path+'//H1//OptionsPivotTables.xlsm'

wb = load_workbook(filename=file_path, read_only=False, keep_vba=True)

ws = wb['OptionsData']  # or wb.active
